<a href="https://colab.research.google.com/github/ErikaNat/Inteligencia-Artificial/blob/main/Ejercicios_Deep_Learning5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import numpy as np
import pandas as pd
import tensorflow as tf

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, StandardScaler, LabelEncoder
from sklearn.compose import ColumnTransformer
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score

In [ ]:
SEED = 42
os.environ["PYTHONHASHSEED"] = str(SEED)
np.random.seed(SEED)
tf.random.set_seed(SEED)
print("Tensorflow:", tf.__version__)

Tensorflow: 2.19.0


In [ ]:
dataset = pd.read_csv("Churn_Modelling.csv")
X = dataset.iloc[:, 3:-1].copy()
y = dataset.iloc[:, -1].values
print("Shapes:", X.shape, y.shape)

Shapes: (10000, 10) (10000,)


In [ ]:
le_gender = LabelEncoder()
X.loc[:, X.columns[2]] = le_gender.fit_transform(X.iloc[:, 2])
ct = ColumnTransformer(
    transformers=[("geo_ohe", OneHotEncoder(handle_unknown="ignore"), [1])],
    remainder="passthrough"
)
X_ohe = ct.fit_transform(X)
X_ohe = np.asarray(X_ohe).astype("float32")

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    X_ohe, y, test_size=0.2, random_state=SEED, stratify=y
)

sc = StandardScaler(with_mean=False)
X_train = sc.fit_transform(X_train).astype("float32")
X_test = sc.transform(X_test).astype("float32")

In [ ]:
from tensorflow.keras import layers, models, callbacks
def build_model(input_dim):
  model = models.Sequential([
      layers.Dense(16, activation="relu", input_shape=(input_dim,)),
      layers.Dense(16, activation="relu"),
      layers.Dense(1, activation="sigmoid")
  ])
  model.compile(
      optimizer="adam",
      loss="binary_crossentropy",
      metrics=["accuracy", tf.keras.metrics.AUC(name="auc")]
  )
  return model
ann = build_model(input_dim=X_train.shape[1])
ann.summary()

/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 16)             │           208 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 16)             │           272 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 1)              │            17 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 497 (1.94 KB)

 Trainable params: 497 (1.94 KB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
cb = [
    callbacks.EarlyStopping(patience=10, restore_best_weights=True, monitor="val_auc", mode="max"),
    callbacks.ModelCheckpoint("best_ann.keras", monitor="val_auc", mode="max", save_best_only=True),
    callbacks.ReduceLROnPlateau(monitor="val_loss", patience=5, factor=0.5)
]

In [ ]:
hists = ann.fit(
    X_train, y_train,
    validation_split=0.2,
    epochs=200,
    batch_size=32,
    callbacks=cb,
    verbose=1
)

Epoch 1/200
200/200 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - accuracy: 0.7883 - auc: 0.5412 - loss: 0.5533 - val_accuracy: 0.8000 - val_auc: 0.6832 - val_loss: 0.4678 - learning_rate: 0.0010
Epoch 2/200
200/200 ━━━━━━━━━━━━━━━━━━━━ 2s 7ms/step - accuracy: 0.7944 - auc: 0.7028 - loss: 0.4692 - val_accuracy: 0.8131 - val_auc: 0.7344 - val_loss: 0.4410 - learning_rate: 0.0010
Epoch 3/200
200/200 ━━━━━━━━━━━━━━━━━━━━ 2s 8ms/step - accuracy: 0.8064 - auc: 0.7505 - loss: 0.4428 - val_accuracy: 0.8206 - val_auc: 0.7532 - val_loss: 0.4284 - learning_rate: 0.0010
Epoch 4/200
200/200 ━━━━━━━━━━━━━━━━━━━━ 2s 10ms/step - accuracy: 0.8179 - auc: 0.7731 - loss: 0.4276 - val_accuracy: 0.8331 - val_auc: 0.7678 - val_loss: 0.4180 - learning_rate: 0.0010
Epoch 5/200
200/200 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.8228 - auc: 0.7875 - loss: 0.4178 - val_accuracy: 0.8369 - val_auc: 0.7783 - val_loss: 0.4101 - learning_rate: 0.0010
Epoch 6/200
200/200 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step - accuracy: 0.8243 - 

In [ ]:
y_proba = ann.predict(X_test).ravel()
y_pred = (y_proba >= 0.5).astype(int)

print("\nMatriz de confusión:\n", confusion_matrix(y_test, y_pred))
print("\nReporte de clasificación:\n", classification_report(y_test, y_pred, digits=4))
print("AUC:", roc_auc_score(y_test, y_proba))


63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step  

Matriz de confusión:
 [[1522   71]
 [ 204  203]]

Reporte de clasificación:
               precision    recall  f1-score   support

           0     0.8818    0.9554    0.9171      1593
           1     0.7409    0.4988    0.5962       407

    accuracy                         0.8625      2000
   macro avg     0.8113    0.7271    0.7567      2000
weighted avg     0.8531    0.8625    0.8518      2000

AUC: 0.856472805625348
